# Linear and Integer Programming

<div align="center">
    <img src="https://raw.githubusercontent.com/psrenergy/QUBO.jl/master/docs/src/assets/logo.svg" width="400px" alt="QUBO.jl">
    <br>
    <a href="#installation">⚙️ Installation Instructions ⚙️<a>
    <br>
    <br>
    <a href="https://colab.research.google.com/github/pedromxavier/QUBO-notebooks/blob/main/julia-template.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</div>

## Introduction to Mathematical Programming

### Modeling
The solution to optimization problems requires the development of a mathematical model. Here we will model an example given in the lecture and see how an integer program can be solved practically.
This example will use as modeling language **[JuMP](http://jump.dev/)**.
This open-source Julia package provides flexible access to different solvers and a general modeling framework for linear and nonlinear integer programs.
The examples solved here will make use of open-source solvers **[GLPK](https://www.gnu.org/software/glpk/)** and **[CLP/CBC](https://projects.coin-or.org/Cbc)** for linear and mixed-integer linear programming, **[IPOPT](https://coin-or.github.io/Ipopt/)** for interior point (non)linear programming, **[BONMIN](https://www.coin-or.org/Bonmin/)** for convex integer nonlinear programming, and **[COUENNE](https://projects.coin-or.org/Couenne)** for nonconvex (global) integer nonlinear programming.

#### Problem statement

Suppose there is a company that produces two different products, A and B, which can be sold at different values, \$5.5 and \$2.1 per unit, respectively.
The company only counts with a single machine with electricity usage of at most 17kW/day. Producing each A and B consumes 8kW/day and 2kW/day, respectively.
Besides, the company can only produce at most 2 more units of B than A per day.

### Linear Programming
This is a valid model, but it would be easier to solve if we had a mathematical representation.
Assuming the units produced of A are $x_1$ and of B are $x_2$ we have

$$
\begin{array}{rl}
    \displaystyle%
    \max_{x_1, x_2} & 5.5x_1 + 2.1x_2 \\
    \textrm{s.t.}   & x_2 \le x_1 + 2 \\
                    & 8x_1 + 2x_2 \le 17 \\
                    & x_1, x_2 \ge 0
\end{array}
$$

In [ ]:
using JuMP

In [ ]:
# Define empty model
lp_model = Model()

# Define the variables
@variable(lp_model, x[1:2] >= 0)

# Define the objective function
@objective(lp_model, Max, 5.5x[1] + 2.1x[2])

# Define the constraints
@constraint(lp_model, c1, x[2] <= x[1] + 2)

@constraint(lp_model, c2, 8x[1] + 2x[2] <= 17)

# Print the model
print(lp_model)

In [ ]:
using GLPK

In [ ]:
# Here we solve the optimization problem with GLPK
set_optimizer(lp_model, GLPK.Optimizer)

set_silent(lp_model)

optimize!(lp_model)

# Display solution of the problem
print(solution_summary(lp_model))

# Retrieve solution value for "x"
x_lp = value.(x)

We observe that the optimal solution of this problem is $x_1 = 1.3$, $x_2 = 3.3$, leading to a profit of $14.08$.

In [ ]:
using Cbc

In [ ]:
# Now with Cbc
set_optimizer(lp_model, Cbc.Optimizer)

set_silent(lp_model)

optimize!(lp_model)

# Display solution of the problem
print(solution_summary(lp_model))

# Retrieve solution value for "x"
x_lp = value.(x)

In [ ]:
using Plots

In [ ]:
# Generate the feasible region plot of this problem
x1 = x2 = range(-0.5, 3.5, 1_000)

# Constraints
c(x1, x2) = (x1 >= 0)      && # Bound: x1 ≥ 0
            (x2 >= 0)      && # Bound: x2 ≥ 0
            (x2 <= x1 + 2) && # Constraint 1: x₂ ≤ x₁ + 2
            (8x1 + 2x2 <= 17) # Constraint 2: 8 x₁ + 2 x₂ ≤ 17

# Objective: min 7.3x₁ + 2.1x₂
z(x1, x2) = 7.3x1 + 2.1x2

# Plot feasible region
plt = heatmap(
    x1,
    x2,
    (x1, x2) -> c(x1, x2) ? z(x1, x2) : NaN;
    legend=:topright,
    xlims=extrema(x1),
    ylims=extrema(x2),
    xlabel=raw"$ x_1 $",
    ylabel=raw"$ x_2 $",
)

# Make plots of constraints
plot!(plt, x1, (x1) -> x1 + 2;
    label=raw"$ x_2 \leq x_1 + 2 $",
    color=:green,
)
plot!(plt, x1, (x1) -> (17 - 8x1) / 2;
    label=raw"$ 8 x_1 + 2 x_2 \leq 17 $",
    color=:blue,
)

# Nonnegativitivy constraints
plot!(plt, zeros(n), x2;
    label=raw"$ x_1 \geq 0 $",
    color=:purple,
)
plot!(plt, x1, zeros(n);
    label=raw"$ x_2 \geq 0 $",
    color=:red,
)

# Optimal solution LP
scatter!(plt, [x_lp[1]], [x_lp[2]];
    label=raw"$ (x_1^\ast, x_2^\ast) $",
    color=:gold
)

The solvers GLPK and CLP implement the simplex method (with many improvements) by default, but we can also use an interior point method through the solver IPOPT (interior point optimizer). IPOPT is able to solve not only linear but also nonlinear problems.

In [ ]:
using Ipopt

In [ ]:
# Using Ipopt for an interior point method
set_optimizer(lp_model, Ipopt.Optimizer)
set_silent(lp_model)

optimize!(lp_model)

print(solution_summary(lp_model))

x_lp = value.(x)

We obtain the same result as previously, but notice that the interior point method reports a solution subject to a certain tolerance, given by its convergence properties when it can get infinitesimally close (but not directly at) the boundary of the feasible region.

## Let's go back to the slides

### Integer Programming

Now let's consider that only integer units of each product can be produced, namely

$$
\begin{array}{rl}
    \displaystyle%
    \max_{x_1, x_2} & 5.5x_1 + 2.1x_2 \\
    \textrm{s.t.}   & x_2 \leq x_1 + 2 \\
                    & 8x_1 + 2x_2 \leq 17 \\
                    & x_1, x_2 \geq 0 \\
                    & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
# Define the model
ilp_model = copy(lp_model)

# Retrieve variable reference
x = ilp_model[:x]

# x₁, x₂ ≥ 0; x₁, x₂ ∈ ℤ
@constraint(ilp_model, [i=1:2], x[i] in MOI.Integer())

# Print the model
print(ilp_model)

In [ ]:
set_optimizer(ilp_model, Cbc.Optimizer)
set_silent(ilp_model)

optimize!(ilp_model)

print(solution_summary(ilp_model))

x_ilp = value.(x)

Here the solution becomes $x_1 = 1, x_2 = 3$ with an objective of $11.8$.

In [ ]:
xi1 = xi2 = 0:4

# Plot relaxed feasible region
plt = heatmap(
    x1,
    x2,
    (x1, x2) -> c(x1, x2) ? z(x1, x2) : NaN;
    legend=:topright,
    xlims=extrema(x1),
    ylims=extrema(x2),
    xlabel=raw"$ x_1 $",
    ylabel=raw"$ x_2 $",
)

# Make plots of constraints
plot!(plt, x1, (x1) -> x1 + 2;
    label=raw"$ x_2 \leq x_1 + 2 $",
    color=:green,
)
plot!(plt, x1, (x1) -> (17 - 8x1) / 2;
    label=raw"$ 8 x_1 + 2 x_2 \leq 17 $",
    color=:blue,
)

# Nonnegativitivy constraints
plot!(plt, zeros(n), x2;
    label=raw"$ x_1 \geq 0 $",
    color=:purple,
)
plot!(plt, x1, zeros(n);
    label=raw"$ x_2 \geq 0 $",
    color=:red,
)

# Feasible solutions
scatter!(plt,
    [xi1[i] for i = 1:5 for j = 1:5 if c(xi1[i], xi2[j])],
    [xi2[j] for i = 1:5 for j = 1:5 if c(xi1[i], xi2[j])];
    label=raw"$ (x_1, x_2) \in \mathbb{Z}^2 $",
    color=:white,
)

# Optimal solution LP
scatter!(plt, [x_ilp[1]], [x_ilp[2]];
    label=raw"$ (x_1^\ast, x_2^\ast) $",
    color=:gold,
)

## Let's go back to the slides

#### Enumeration
Enumerating all the possible solutions in this problem might be very efficient (there are only 8 feasible solutions), this we only know from the plot. Assuming that we had as upper bounds for the variables 4, the possible solutions would be 16. With a larger number of variables, the enumeration turns to be impractical. For $n$ binary variables (we can always "binarize" the integer variables), the number of possible solutions is $2^n$.

In many other applications, the possible solutions come from permutations of the integer variables (e.g., assignment problems), which grow as $n!$ with the size of the input.

This growth makes the problem grow out of control relatively quickly.

In [ ]:
import SpecialFunctions

In [ ]:
i = 1:100

plt = plot(;
    title=raw"Orders of Magnitude for $ n! $ and $ 2^n $",
    xlabel=raw"$ n $",
    legend=:topleft,
    yscale=:log10,
)

plot!(plt, i, (i) -> 2.0^i; label=raw"$ 2^n $", color=:blue)
plot!(plt, i, SpecialFunctions.gamma; label=raw"$ n! $", color=:red)

plot!(plt, i, (i) -> 3.154E16; color=:gray, label = nothing)
annotate!(plt, first(i), 3.154E16, text("ns in a year", :gray, :left, :bottom, 7))

plot!(plt, i, (i) -> 4.3E26; color=:gray, label = nothing)
annotate!(plt, first(i), 4.3E26, text("age of the universe in ns", :gray, :left, :bottom, 7))

plot!(plt, i, (i) -> 6E79; color=:gray, label = nothing)
annotate!(plt, first(i), 6E79, text("atoms in the universe", :gray, :left, :bottom, 7))

## Let's go back to the slides

### Integer convex nonlinear programming

The following constraint: "the production of B minus 1, squared, can only be smaller than 2 minus the production of A" can be incorporated  in the following convex integer nonlinear program, 

$$
\begin{array}{rl}
    \displaystyle%
    \max_{x_1, x_2} & 5.5x_1 + 2.1x_2        \\
    \textrm{s.t.}   & x_2 \leq x_1 + 2       \\
                    & 8x_1 + 2x_2 \leq 17    \\
                    & (x_2-1)^2 \leq 2 - x_1 \\
                    & x_1, x_2 \geq 0        \\
                    & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
# Define the model
cvxnl_model = copy(ilp_model)

# Retrieve variable reference
x = cvxnl_model[:x]

# (x₂ - 1)² ≤ 2 - x₁
@constraint(cvxnl_model, c3, (x[2] - 1)^2 <= 2 - x[1])

# Print the model
print(cvxnl_model)

In [ ]:
using AmplNLWriter
using Bonmin_jll

const Bonmin_Optimizer = () -> AmplNLWriter.Optimizer(Bonmin_jll.amplexe);

In [ ]:
set_optimizer(cvxnl_model, Bonmin_Optimizer)

optimize!(cvxnl_model)

print(solution_summary(cvxnl_model))

x_cvxnl = value.(x)

In [ ]:
χ(x1, x2) = c(x1, x2) && ((x2 - 1)^2 <= 2 - x1) # Constraint 3: (x₂ - 1)² ≤ 2 - x₁

# Generate the feasible region plot of this problem
n = 1_000

xi1 = xi2 = 0:4

# Plot relaxed feasible region
plt = heatmap(
    x1,
    x2,
    (x1, x2) -> χ(x1, x2) ? z(x1, x2) : NaN;
    legend=:topright,
    xlims=extrema(x1),
    ylims=extrema(x2),
    xlabel=raw"$ x_1 $",
    ylabel=raw"$ x_2 $",
)

# Make plots of constraints
plot!(plt, x1, (x1) -> x1 + 2;
    label=raw"$ x_2 \leq x_1 + 2 $",
    color=:green,
)
plot!(plt, x1, (x1) -> (17 - 8x1) / 2;
    label=raw"$ 8 x_1 + 2 x_2 \leq 17 $",
    color=:blue,
)
plot!(plt, 2 .- (x1 .- 1) .^ 2, x1,;
    label=raw"$ (x_2 - 1)^2 \leq 2 - x_1 $",
    color=:orange,
)

# Nonnegativitivy constraints
plot!(plt, zeros(n), x2;
    label=raw"$ x_1 \geq 0 $",
    color=:purple,
)
plot!(plt, x1, zeros(n);
    label=raw"$ x_2 \geq 0 $",
    color=:red,
)

# Feasible solutions
scatter!(plt,
    [xi1[i] for i = 1:5 for j = 1:5 if χ(xi1[i], xi2[j])],
    [xi2[j] for i = 1:5 for j = 1:5 if χ(xi1[i], xi2[j])];
    label=raw"$ (x_1, x_2) \in \mathbb{Z}^2 $",
    color=:white,
)

# Optimal solution LP
scatter!(plt, [x_cvxnl[1]], [x_cvxnl[2]];
    label=raw"$ (x_1^\ast, x_2^\ast) $",
    color=:gold,
)

In this case the optimal solution becomes $x_1 = 1, x_2 = 2$ with an objective of $9.7$.

### Integer non-convex programming

The last constraint "the production of B minus 1 squared can only be greater than the production of A plus one half" can be incorporated  in the following convex integer nonlinear program

$$
\begin{array}{rl}
    \displaystyle%
    \max_{x_1, x_2} & 5.5x_1 + 2.1x_2 \\
    \textrm{s.t.}   & x_2 \leq x_1 + 2 \\
                    & 8x_1 + 2x_2 \leq 17 \\
                    & (x_2-1)^2 \leq 2-x_1\\
                    & (x_2-1)^2 \geq \frac{1}{2} +x_1\\
                    & x_1, x_2 \geq 0 \\
                    & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
# Define the model
ncvxnl_model = copy(cvxnl_model)

# Retrieve variable reference
x = ncvxnl_model[:x]

# (x₂ - 1)² ≥ 1/2 + x₁
@constraint(ncvxnl_model, c4, (x[2] - 1)^2 >= 1/2 + x[1])

# Print the model
print(ncvxnl_model)

In [ ]:
# Trying to solve the problem with BONMIN we might obtain the optimal solution,
# but we have no guarantees
set_optimizer(ncvxnl_model, Bonmin_Optimizer)

optimize!(ncvxnl_model)

print(solution_summary(ncvxnl_model))

if result_count(ncvxnl_model) > 0
    x_ncvxnl = value.(x)
end

In [ ]:
using Couenne_jll

const Couenne_Optimizer = () -> AmplNLWriter.Optimizer(Couenne_jll.amplexe);

In [ ]:
# Trying to solve the problem with global MINLP solver COUENNE
set_optimizer(ncvxnl_model, Couenne_Optimizer)

optimize!(ncvxnl_model)

print(solution_summary(ncvxnl_model))

if result_count(ncvxnl_model) > 0
    x_ncvxnl = value.(x)
end

In [ ]:
ξ(x1, x2) = χ(x1, x2) && ((x2 - 1)^2 >= 1/2 + x1) # Constraint 4: (x₂ - 1)² ≤ 1/2 + x₁

# Generate the feasible region plot of this problem
n = 1_000

xi1 = xi2 = 0:4

# Plot relaxed feasible region
plt = heatmap(
    x1,
    x2,
    (x1, x2) -> ξ(x1, x2) ? z(x1, x2) : NaN;
    legend=:topright,
    xlims=extrema(x1),
    ylims=extrema(x2),
    xlabel=raw"$ x_1 $",
    ylabel=raw"$ x_2 $",
)

# Make plots of constraints
plot!(plt, x1, (x1) -> x1 + 2;
    label=raw"$ x_2 \leq x_1 + 2 $",
    color=:green,
)
plot!(plt, x1, (x1) -> (17 - 8x1) / 2;
    label=raw"$ 8 x_1 + 2 x_2 \leq 17 $",
    color=:blue,
)
plot!(plt, 2 .- (x1 .- 1) .^ 2, x1,;
    label=raw"$ (x_2 - 1)^2 \leq 2 - x_1 $",
    color=:orange,
)
plot!(plt, -1/2 .+ (x1 .- 1) .^ 2, x1,;
    label=raw"$ (x_2 - 1)^2 \leq \frac{1}{2} + x_1 $",
    color=:magenta,
)

# Nonnegativitivy constraints
plot!(plt, zeros(n), x2;
    label=raw"$ x_1 \geq 0 $",
    color=:purple,
)
plot!(plt, x1, zeros(n);
    label=raw"$ x_2 \geq 0 $",
    color=:red,
)

# Feasible solutions
scatter!(plt,
    [xi1[i] for i = 1:5 for j = 1:5 if ξ(xi1[i], xi2[j])],
    [xi2[j] for i = 1:5 for j = 1:5 if ξ(xi1[i], xi2[j])];
    label=raw"$ (x_1, x_2) \in \mathbb{Z}^2 $",
    color=:white,
)

# Optimal solution LP
scatter!(plt, [x_ncvxnl[1]], [x_ncvxnl[2]];
    label=raw"$ (x_1^\ast, x_2^\ast) $",
    color=:gold,
)

We are able to solve non-convex MINLP problems. However, the complexity of these problems leads to significant computational challenges that need to be tackled.

### Powerful commercial solver installation

#### Gurobi Installation

Gurobi is one of the most powerful LP and MIP solvers available today. They provide free academic licenses. In order to install the software, visit their **[Website](https://www.gurobi.com/)**, create an account (preferably with your CMU email), and obtain a license. Once you do that, you can download and use the software.

#### BARON Installation

BARON is one of the most powerful MINLP solvers available today. CMU students are given a free license given by the association of BARON's developer (Prof. Nick Sahinidis) to CMU. In order to install the software, visit their **[Website](https://www.minlp.com/home)**, create an account (with your CMU email), and obtain a license. Once you do that you can download and use the software.

# References

- [Julia Colab Notebook Template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb)
- [QuIPML22](https://github.com/bernalde/QuIPML22/blob/main/notebooks/Notebook%201%20-%20LP%20and%20IP.ipynb)

# Installation

## Colab Instructions

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia and IJulia. This can take a couple of minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell

bash <(curl -s "https://github.com/pedromxavier/QUBO-notebooks/blob/main/scripts/install-colab-julia.sh")

### Validate Installation

In [ ]:
versioninfo()

## Julia Packages

In [ ]:
import Pkg

Pkg.activate(; temp=true)

Pkg.add.([
    "JuMP",
    "GLPK",
    "Cbc",
    "Ipopt",
    "AmplNLWriter",
    "Bonmin_jll",
    "Couenne_jll",
    "Plots",
    "SpecialFunctions",
]; io=devnull); # Suppress Output

<div align="center">
    <a href="#">🔝 Go back to the top 🔝</a>
</div>